In [1]:
# Based on: https://www.tensorflow.org/lite/guide/python
# And then https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/examples/python/

# If running on colab - you'll need to attach
from google.colab import drive
drive.mount('/$USER/tensorflow_datasets')

Build (if necessary) and then import the british birds dataset...

PRO-TIP:
Adding to this if you’re worried if the colab could be disconnecting you can run a small following Java script code going to the console by clicking Ctrl+ Shift + i in your browser.

    function ClickConnect(){
      console.log("Working"); 
      document.querySelector("colab-toolbar-button#connect").click() 
    }setInterval(ClickConnect,60000)


In [12]:
# If you haven't build the dataset, you need to go there and do that first...
#########################
# THIS WILL DOWNLOAD ALL THE DATA - YOU ONLY NEED TO RUN THIS ONCE
# If you do this in Colab - make sure you have the space by mounting drive!
#########################
!tfds build ../datasets/britishbirds

# You can then...
#import sys
#sys.path.append("../")
#import datasets.britishbirds
#
#ds = tfds.load('britishbirds')  # `my_dataset` registered

INFO[build.py]: Loading dataset ../datasets/britishbirds from path: /Users/tim/Code/github/birdsong/datasets/britishbirds/britishbirds.py
2022-02-21 21:36:19.681225: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
INFO[build.py]: download_and_prepare for dataset britishbirds/1.0.0...
INFO[dataset_builder.py]: Generating dataset britishbirds (/Users/tim/tensorflow_datasets/britishbirds/1.0.0)
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

                                        ? file/s]

                                           
INFO[download_manager.py]: Downloading h

Extraction completed...: 0 file [01:49, ? file/s]
Dl Size...:   4%|█▏                          | 30/712 [01:53<54:58,  4.84s/ MiB]

Extraction completed...: 0 file [01:53, ? file/s]
Dl Size...:   4%|█▏                          | 31/712 [01:56<49:04,  4.32s/ MiB]

Extraction completed...: 0 file [01:56, ? file/s]
Dl Size...:   4%|█▎                          | 32/712 [02:04<59:57,  5.29s/ MiB]

Extraction completed...: 0 file [02:04, ? file/s]
Dl Size...:   5%|█▏                        | 33/712 [02:11<1:05:26,  5.78s/ MiB]

Extraction completed...: 0 file [02:11, ? file/s]
Dl Size...:   5%|█▏                        | 34/712 [02:15<1:00:03,  5.32s/ MiB]

Extraction completed...: 0 file [02:15, ? file/s]
Dl Size...:   5%|█▍                          | 35/712 [02:17<48:13,  4.27s/ MiB]

Extraction completed...: 0 file [02:17, ? file/s]
Dl Size...:   5%|█▍                          | 36/712 [02:19<42:38,  3.78s/ MiB]

Extraction completed...: 0 file [02:19, ? file/s]
Dl Size...:   5%|█▍       

Extraction completed...: 0 file [05:36, ? file/s]
Dl Size...:   9%|██▍                       | 67/712 [05:41<1:07:33,  6.28s/ MiB]

Extraction completed...: 0 file [05:41, ? file/s]
Dl Size...:  10%|██▍                       | 68/712 [05:49<1:12:32,  6.76s/ MiB]

Extraction completed...: 0 file [05:49, ? file/s]
Dl Size...:  10%|██▌                       | 69/712 [05:56<1:13:56,  6.90s/ MiB]

Extraction completed...: 0 file [05:56, ? file/s]
Dl Size...:  10%|██▌                       | 70/712 [06:05<1:18:52,  7.37s/ MiB]

Extraction completed...: 0 file [06:05, ? file/s]
Dl Size...:  10%|██▌                       | 71/712 [06:15<1:27:44,  8.21s/ MiB]

Extraction completed...: 0 file [06:15, ? file/s]
Dl Size...:  10%|██▋                       | 72/712 [06:22<1:23:53,  7.87s/ MiB]

Extraction completed...: 0 file [06:22, ? file/s]
Dl Size...:  10%|██▋                       | 73/712 [06:31<1:27:57,  8.26s/ MiB]

Extraction completed...: 0 file [06:31, ? file/s]
Dl Size...:  10%|██▋      

In [10]:
from carbontracker.tracker import CarbonTracker

tracker = CarbonTracker(epochs=max_epochs)

#!pip3 install --index-url https://google-coral.github.io/py-repo/ tflite_runtime

ModuleNotFoundError: No module named 'carbontracker'

In [3]:
# Get photo
!curl https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/examples/label_image/testdata/grace_hopper.bmp > /tmp/grace_hopper.bmp
# Get model
!curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_1.0_224.tgz | tar xzv -C /tmp
# Get labels
!curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_1.0_224_frozen.tgz  | tar xzv -C /tmp  mobilenet_v1_1.0_224/labels.txt

!mv /tmp/mobilenet_v1_1.0_224/labels.txt /tmp/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  918k  100  918k    0     0  3506k      0 --:--:-- --:--:-- --:--:-- 3506k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0./
./mobilenet_v1_1.0_224.tflite
  5 89.9M    5 4904k    0     0  6582k      0  0:00:13 --:--:--  0:00:13 6573k./mobilenet_v1_1.0_224.ckpt.meta
./mobilenet_v1_1.0_224.ckpt.index
./mobilenet_v1_1.0_224.ckpt.data-00000-of-00001
 26 89.9M   26 24.0M    0     0  13.8M      0  0:00:06  0:00:01  0:00:05 13.8M./mobilenet_v1_1.0_224_info.txt
./mobilenet_v1_1.0_224_frozen.pb
100 89.9M  100 89.9M    0     0  31.5M      0  0:00:02  0:00:02 --:--:-- 31.4M
./mobilenet_v1_1.0_224_eval.pbtxt
  % Total    % Received % Xferd  Average Spee

In [4]:
model_file = "/tmp/mobilenet_v1_1.0_224.tflite"
label_file = "/tmp/labels.txt"
image_file = "/tmp/grace_hopper.bmp"
input_mean = 127.5
input_std = 127.5

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import time

import numpy as np
from PIL import Image

import tflite_runtime.interpreter as tflite

def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]


interpreter = tflite.Interpreter(model_path=model_file)

interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
img = Image.open(image_file).resize((width, height))

# add N dim
input_data = np.expand_dims(img, axis=0)

if floating_model:
  input_data = (np.float32(input_data) - input_mean) / input_std

interpreter.set_tensor(input_details[0]['index'], input_data)

start_time = time.time()
interpreter.invoke()
stop_time = time.time()

output_data = interpreter.get_tensor(output_details[0]['index'])
results = np.squeeze(output_data)

top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)
for i in top_k:
  if floating_model:
    print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
  else:
    print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))

0.919721: 653:military uniform
0.017762: 907:Windsor tie
0.007507: 668:mortarboard
0.005419: 466:bulletproof vest
0.003828: 458:bow tie, bow-tie, bowtie
time: 113.866ms


In [8]:
# Save the model
import tensorflow as tf

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
tflite_model = converter.convert()

# Save the model to disk
open("sine_model.tflite", "wb").write(tflite_model)

# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
# Indicate that we want to perform the default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Define a generator function that provides our test data's x values
# as a representative dataset, and tell the converter to use it
def representative_dataset_generator():
  for value in x_test:
    # Each scalar value must be inside of a 2D array that is wrapped in a list
    yield [np.array(value, dtype=np.float32, ndmin=2)]
converter.representative_dataset = representative_dataset_generator
# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open("sine_model_quantized.tflite", "wb").write(tflite_model)

NameError: ignored